In [1]:
#imports
import pandas as pd
import numpy as np
import gurobipy as gp

In [2]:
#teacher evaluation --> 1 - Improvement required (not the sharpest pencil), 10 - Excellent, per subject
subject_and_teacher, teacher_evaluation = gp.multidict({
    'Bible_Roni': 5,
    'Bible_Rachel': 5,
    'Literature_Sharon': 6,
    'Literature_Liora': 6,
    'Math_Yuri': 3,
    'Math_Sarit': 3,
    'Math_Victor': 3,
    'English_Miriam': 7,
    'English_Haim': 7,
    'Physics_Tzvika': 2,
    'Language_Hanita': 4,
    'Language_Smadar': 4,
    'Geography_Ilana': 1,
    'History_Ruth': 3,
    'History_Etti': 3,
    'Chemistry_Shelly': 7,
    'CitizenshipLesson_Tami': 8,
    'Economics_Menachem': 5,
    'Arabic_Hanita': 9
})

In [3]:
#subject + avg grade
student_avg = gp.multidict({
    'Bible_Roni': 70,
    'Bible_Rachel': 70,
    'Literature_Sharon': 72,
    'Literature_Liora': 72,
    'Math_Yuri': 53,
    'Math_Sarit': 53,
    'Math_Victor': 53,
    'English_Miriam': 68,
    'English_Haim': 68,
    'Physics_Tzvika': 41,
    'Language_Hanita': 80,
    'Language_Smadar': 80,
    'Geography_Ilana': 94,
    'History_Ruth': 78,
    'History_Etti': 78,
    'Chemistry_Shelly': 75,
    'CitizenshipLesson_Tami': 86,
    'Economics_Menachem': 56,
    'Arabic_Hanita': 66
})[1]

In [4]:
teacher_free_time = {
    'Sunday': [['Bible_Rachel', 10, 12], ['Literature_Liora', 14, 16]],
    'Monday': [['Literature_Sharon', 13, 15], ['Math_Yuri', 12, 14], ['English_Haim', 17, 19]],
    'Tuesday': [['Physics_Tzvika', 16, 18], ['Language_Hanita', 14, 16],
                ['Economics_Menachem', 14, 16], ['English_Miriam', 10, 12]],
    'Wednesday': [['Language_Smadar', 9, 11], ['Geography_Ilana', 13, 15],
                  ['History_Ruth', 12, 14], ['CitizenshipLesson_Tami', 15, 17]],
    'Thursday': [['Math_Sarit', 9, 11], ['Math_Victor', 17, 19], ['Arabic_Hanita', 18, 20]],
    'Friday': [['Bible_Roni', 8, 10], ['History_Etti', 10, 12], ['Chemistry_Shelly', 11, 13]]
}

In [5]:
student_free_time = {
    'Sunday': [14,20],
    'Monday': [12,15],
    'Tuesday': [16,19],
    'Wednesday': [12,17],
    'Thursday': [17,20],
    'Friday': [8,13]
}

In [6]:
#find the subjects which their time collides while checking the days and houres
#teacher_free_time is when the classes occurs
def get_collisions(teacher_free_time):
    collisions = []
    for subjects_per_day in teacher_free_time.values():
        # subjects_per_day is a list of all the subjects in each day
        if len(subjects_per_day) == 1:
            continue  # there are no possible collisions because there is only one subject a day.

        # in case there is more than one subjects per day, check if the class times collides by comparing start and end times.
        for i, course in enumerate(subjects_per_day):
            if i == len(subjects_per_day) - 1:
                continue  # all collisions were already specified before
            for other_course in subjects_per_day[i + 1:]:
                start_time_current = course[1]
                end_time_current = course[2]
                start_time_other = other_course[1]
                course_name = course[0]
                other_course_name = other_course[0]
                if (start_time_other < end_time_current) or (start_time_other == start_time_current):
                    collisions.append((course_name, other_course_name))

    return collisions

collisions = get_collisions(teacher_free_time)

In [7]:
print(collisions)

[('Literature_Sharon', 'Math_Yuri'), ('Physics_Tzvika', 'Language_Hanita'), ('Physics_Tzvika', 'Economics_Menachem'), ('Physics_Tzvika', 'English_Miriam'), ('Language_Hanita', 'Economics_Menachem'), ('Language_Hanita', 'English_Miriam'), ('Economics_Menachem', 'English_Miriam'), ('Geography_Ilana', 'History_Ruth'), ('Math_Victor', 'Arabic_Hanita'), ('History_Etti', 'Chemistry_Shelly')]


In [8]:
#find the subjects which their time does not match with the student schedule
def get_inopportune_classes(teacher_free_time, student_free_time):
    #a list to hold all the classes that not fit with the student schedule
    inopportune_classes = []
    #loop over the two dictinaries at once
    for (student_day, s_free_time), (day, day_subjects) in zip(student_free_time.items(), teacher_free_time.items()):
        day_length = student_free_time[student_day]     
        for subject in day_subjects:
            start = subject[1]
            end = subject[2]
            if start < day_length[0] or end > day_length[1]:
                inopportune_classes.append(subject[0])

    return inopportune_classes

inopportune_classes = get_inopportune_classes(teacher_free_time, student_free_time)

In [9]:
print(inopportune_classes)

['Bible_Rachel', 'English_Haim', 'Language_Hanita', 'Economics_Menachem', 'English_Miriam', 'Language_Smadar', 'Math_Sarit']


In [10]:
#some subjects are the same with different teachers (like bible, math..)
#we want the student to be able to enroll to a subject only once (and ont once per teacher).
def get_same_classes(teacher_free_time):
    same_subjects = {}
    for day in teacher_free_time.values():
        #for example: subject --> ['Bible_Rachel', 10, 12]
        for subject in day:
            #subject[0] --> Bible_Rachel
            split_sub_and_teacher = subject[0].split("_")
            #split_sub_and_teacher --> Bible,Rachel
            if split_sub_and_teacher[0] in same_subjects:
                same_subjects[split_sub_and_teacher[0]].append(subject)
            else:
                same_subjects[split_sub_and_teacher[0]] = [subject]
    return same_subjects

same_subjects = get_same_classes(teacher_free_time)

In [11]:
print(same_subjects)

{'Bible': [['Bible_Rachel', 10, 12], ['Bible_Roni', 8, 10]], 'Literature': [['Literature_Liora', 14, 16], ['Literature_Sharon', 13, 15]], 'Math': [['Math_Yuri', 12, 14], ['Math_Sarit', 9, 11], ['Math_Victor', 17, 19]], 'English': [['English_Haim', 17, 19], ['English_Miriam', 10, 12]], 'Physics': [['Physics_Tzvika', 16, 18]], 'Language': [['Language_Hanita', 14, 16], ['Language_Smadar', 9, 11]], 'Economics': [['Economics_Menachem', 14, 16]], 'Geography': [['Geography_Ilana', 13, 15]], 'History': [['History_Ruth', 12, 14], ['History_Etti', 10, 12]], 'CitizenshipLesson': [['CitizenshipLesson_Tami', 15, 17]], 'Arabic': [['Arabic_Hanita', 18, 20]], 'Chemistry': [['Chemistry_Shelly', 11, 13]]}


In [12]:
# change the vars to match with our goals - min grades and min evaluation
def adjust_the_vars(teacher_evaluation, student_avg, max_eva, max_avg):
    
    for i in teacher_evaluation:
         teacher_evaluation[i] = max_eva - teacher_evaluation[i]
    for j in student_avg:
        student_avg[j] = max_avg - student_avg[j]
    
    print(teacher_evaluation)
    print(student_avg)

adjust_the_vars(teacher_evaluation, student_avg, 10, 100)

{'Bible_Roni': 5, 'Bible_Rachel': 5, 'Literature_Sharon': 4, 'Literature_Liora': 4, 'Math_Yuri': 7, 'Math_Sarit': 7, 'Math_Victor': 7, 'English_Miriam': 3, 'English_Haim': 3, 'Physics_Tzvika': 8, 'Language_Hanita': 6, 'Language_Smadar': 6, 'Geography_Ilana': 9, 'History_Ruth': 7, 'History_Etti': 7, 'Chemistry_Shelly': 3, 'CitizenshipLesson_Tami': 2, 'Economics_Menachem': 5, 'Arabic_Hanita': 1}
{'Bible_Roni': 30, 'Bible_Rachel': 30, 'Literature_Sharon': 28, 'Literature_Liora': 28, 'Math_Yuri': 47, 'Math_Sarit': 47, 'Math_Victor': 47, 'English_Miriam': 32, 'English_Haim': 32, 'Physics_Tzvika': 59, 'Language_Hanita': 20, 'Language_Smadar': 20, 'Geography_Ilana': 6, 'History_Ruth': 22, 'History_Etti': 22, 'Chemistry_Shelly': 25, 'CitizenshipLesson_Tami': 14, 'Economics_Menachem': 44, 'Arabic_Hanita': 34}


In [13]:
import time
import datetime
start = time.time()
start = datetime.datetime.now()

# Creating the model
model = gp.Model("Partani")

# Create decision variables for the subjects the student should take as 'partani' houres.
subjects_to_assign = model.addVars(subject_and_teacher, name="Subjects+Teachers", vtype=gp.GRB.BINARY)

Restricted license - for non-production use only - expires 2022-01-13


In [14]:
# objective
# The objective is to maximize (take as many subjects with the lowest grades and teacher evaluation)

model.ModelSense = gp.GRB.MAXIMIZE

evaluation_objective = subjects_to_assign.prod(teacher_evaluation)
evaluation_priority = 1
evaluation_weight = 0.5

student_avg_objective = subjects_to_assign.prod(student_avg)/10
student_avg_priority = 0
student_avg_weight = 1

model.setObjectiveN(evaluation_objective, evaluation_priority, weight=evaluation_weight)
model.setObjectiveN(student_avg_objective, student_avg_priority, weight=student_avg_weight)

In [15]:
def add_constrains_to_model(subjects_to_assign, collisions, inopportune_classes, same_subjects):

    for course1, course2 in collisions:
        model.addConstr(subjects_to_assign.sum(course1) + subjects_to_assign.sum(course2) <= 1)

    for course in inopportune_classes:
        model.addConstr(subjects_to_assign.sum(course) == 0)

    for course in same_subjects.values():
        courses_sum = 0
        for course_time in course:
            courses_sum += subjects_to_assign.sum(course_time)
        model.addConstr(courses_sum <= 1)

In [16]:
def optimize(subjects_to_assign, collisions, inopportune_classes, same_subjects):
    add_constrains_to_model(subjects_to_assign, collisions, inopportune_classes, same_subjects)
    model.optimize()
    end = time.time()
    end = datetime.datetime.now()
    print(end - start)
    
optimize(subjects_to_assign, collisions, inopportune_classes, same_subjects)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 29 rows, 19 columns and 46 nonzeros
Model fingerprint: 0x131dea92
Variable types: 0 continuous, 19 integer (19 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 29 rows, 19 columns and 46 nonzeros
Variable types: 0 continuous, 19 integer (19 binary)
Coefficient statistic

In [17]:
def schedule_partani(teacher_free_time, teacher_evaluation):
    teacher_eval_min_score = 0
    for day, data in teacher_free_time.items():
        print(':',day)
        for subj in data:
            for v in model.getVars():
                split_teacher = v.varName.split("[")
                split_again = split_teacher[1].split("]")
                if subj[0] == split_again[0]:
                    if v.x ==1:
                        print('%s' % (split_again[0]),subj[1:])
                        teacher_eval_min_score+= (10-teacher_evaluation[split_again[0]])
        print("_______________________________")
    return teacher_eval_min_score

min_evaluation = schedule_partani(teacher_free_time, teacher_evaluation)

: Sunday
Literature_Liora [14, 16]
_______________________________
: Monday
Math_Yuri [12, 14]
_______________________________
: Tuesday
Physics_Tzvika [16, 18]
_______________________________
: Wednesday
Geography_Ilana [13, 15]
CitizenshipLesson_Tami [15, 17]
_______________________________
: Thursday
Arabic_Hanita [18, 20]
_______________________________
: Friday
Bible_Roni [8, 10]
History_Etti [10, 12]
_______________________________


In [18]:
print("The minimum Teacher Evaluation Score is:", min_evaluation, "Out of 120")

The minimum Teacher Evaluation Score is: 37 Out of 120
